In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf

import datetime
import time
from datetime import timedelta
import os

%matplotlib inline

In [2]:
df = pd.read_csv(r"C:\Users\rg654th\Downloads\Flight Delay\Clean Data\Clean Historical Data.csv", parse_dates=[0], infer_datetime_format=True)

In [3]:
# New Column and Format
df['Scheduled DateTime'] = pd.to_datetime(df['Scheduled DateTime'], infer_datetime_format=True)
df['Delay? (15 min)'] = np.where(df['Minute Delay'] >= 15, 'Yes', 'No')

In [4]:
# df['Sched_mins_since_midnight']
df['Sched_mins_since_midnight'] = (df['Scheduled DateTime'] - df['Scheduled DateTime'].apply(lambda dt: dt.replace(hour=0, minute=0, second=0, microsecond=0)))
df['Sched_mins_since_midnight'] = df['Sched_mins_since_midnight'] / np.timedelta64(1, 'm')

In [5]:
traindf = df.drop(['Flight No.', 'Scheduled Date','Scheduled DateTime', 'Scheduled Time', 'Real Departure Time'], axis=1)
traindf = traindf.dropna()
traindf.shape

(3921, 9)

In [6]:
newtraindf = pd.get_dummies(traindf)
newtraindf.head(1)

,Minute Delay,Sched_mins_since_midnight,Terminal_T1,Terminal_T2,Destination_Aberdeen,Destination_Abu Dhabi,Destination_Agadir,Destination_Alicante,Destination_Almeria,Destination_Amsterdam,...,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Delay Type_Long,Delay Type_Medium,Delay Type_None,Delay Type_Short,Delay? (15 min)_No,Delay? (15 min)_Yes
0,0,980.0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [12]:
# Split Data
x_data = newtraindf.drop(['Delay Type_None', 'Delay Type_Short', 'Delay Type_Long', 'Delay Type_Medium', 'Delay? (15 min)_No', 'Delay? (15 min)_Yes'],axis=1)
y_data = newtraindf['Minute Delay']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.2)

In [13]:
# SGD Regressor, Lasso, Elastic Net, Ridge Regression, SVR(kernel = 'linear'or'rbf'), Ensemble
from sklearn.linear_model import SGDRegressor

In [19]:
model = SGDRegressor()
model.fit(X_train, y_train)

C:\Users\rg654th\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=5, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [20]:
y_pred = model.predict(X_test)

score2 = metrics.r2_score(y_test, y_pred)
score = metrics.mean_squared_error(y_test, y_pred)
print("MSE","=", score)
print("R-Squared","=", score2)

MSE = 3.746903737811043e+29
R-Squared = -7.607331765643255e+26
